In [1]:
import torch
import nbsetup 

# Diffusions and Convergence

In this notebook, we assess the convergence benefits brought over the Generalised Belief Propagation algorithm 
by the GBP and Bethe $\varepsilon$-diffusions [[1][gsi21]], for estimating marginals of graphical models on a hypergraph $(\Omega, K)$ of random variables, i.e. probability distributions $p_\Omega$ on a joint variable $x_\Omega = (x_i)_{i \in \Omega}$ that factorise as a product of local factors $f_\alpha$ for $\alpha \subseteq \Omega$:

$$ p_\Omega(x_\Omega) = \frac 1 {Z_\Omega} \prod_{\alpha \in K} f_\alpha (x_\alpha) 
= \frac 1 {Z_\Omega} {\rm e}^{- \sum_\alpha u_\alpha(x_\alpha)} 
$$

## System

The chosen hypergraph $(\Omega, K)$ is the $\cap$-closure of three triangles $ijk$, $ikl$ and $jkl$. This is the simplest hypergraph for which GBP does not surely converge to a unique equilibrium. The `System` constructor receives an iterable of strings of the form `i:j:k` to represent faces $\alpha \in K$ of the hypergraph, i.e. collections $\alpha \subset \Omega$ of vertices. 

[gsi21]: https://opeltre.github.io/bib/gsi21.pdf

In [2]:
from system import System

K = System(('i:j:k', 'j:k:l', 'i:k:l'), shape=2)

# vertices:
print(f"\u03A9: {K.vertices()}")
# faces:
print(f"K: {K}")

Ω: (i:j:k:l)
K: ((i:j:k),(i:k),(i:k:l),(j:k),(j:k:l),(k),(k:l))


![factor graph for the 2-horn](img/GM.svg)

The returned `System` instance `K` constructs $n$ sets of keys `K[0], ..., K[n]` representing the _nerve_ of the partial order $(K, \supseteq)$. 

In [3]:
# 2-nerve:
for chain in K[2]:
    print(chain)

(j:k:l) . (k:l) . (k)
(j:k:l) . (j:k) . (k)
(i:j:k) . (i:k) . (k)
(i:j:k) . (j:k) . (k)
(i:k:l) . (i:k) . (k)
(i:k:l) . (k:l) . (k)


The `shape` attribute describes possible values of each individual degree of freedom $x_i \in E_i$ for all $i \in \Omega$. The faces $\alpha \subseteq \Omega$ of $K$ are consequently assigned a local configuration space $E_\alpha = \prod_{i \in \alpha} E_i$. The `shape` parameter can be passed as a dictionary of atomic shapes indexed by vertices. Systems are binary by default, other constant integer values `shape=n` being interpreted as $E_i = \{0, \dots, n-1\}$ for all $i$. 


## Fields 

Degree-$p$ _fields_ $\varphi \in A_p$ are represented by dictionaries mapping every chain $\alpha_0\dots \alpha_p$ in `K[p]` to a tensor representing a local function on $E_{\alpha_p}$: 

$$ A_p = \prod_{\alpha_0 \supset \dots \supset \alpha_p} {\mathbb R}^{E_{\alpha_p}} 
= \prod_{\alpha_0 \supset \dots \supset \alpha_p} A_{\alpha_p} $$

Degree-0 fields or _potentials_ $u \in A_0$ are therefore collections of local functions $u_\alpha(x_\alpha)$ indexed by faces $\alpha$ of $K$. 

Degree-1 fields or _fluxes_ $\varphi \in A_1$ are collections of local functions $\varphi_{\alpha\beta}(x_\beta)$ 
indexed by pairs $\alpha \supset \beta$ in $K$. 

Degree-$p$ fields can be created by calling `K.gaussian(p)` or `K.zeros(p)`, which return `Tensor` instances.

In [4]:
# 2-Field:
K.gaussian(2)

Tensor {
(i:j:k) . (i:k) . (k) :-> tensor([0.1599, 0.6350])
(i:j:k) . (j:k) . (k) :-> tensor([2.0954, 1.2959])
(i:k:l) . (i:k) . (k) :-> tensor([1.8487, 0.3176])
(i:k:l) . (k:l) . (k) :-> tensor([-0.4168,  0.8201])
(j:k:l) . (j:k) . (k) :-> tensor([-0.0009, -0.3842])
(j:k:l) . (k:l) . (k) :-> tensor([-0.5940,  0.2935])
}

Natural operators on $K$ are also represented as higher dimensional tensors: 

### differential and codifferential

$-$ The two adjoint _differentials_ $d = \delta^*$ for the canonical metric on $A_\bullet$ are returned `K.d(n)` and `K.delta(n + 1)`, where $d^{(n)} : A_n \to  A_{n+1}$ elevates and $\delta^{(n+1)} : A_{n+1} \to A_n$ lowers field degrees. They satisfy $d \circ d = 0 = \delta \circ \delta$ as operators on the full complex $A_\bullet = \oplus_n A_n$. The degree $0$ differential $d$ acts on a density $q \in A_0$ by: 

$$ (q_\alpha) \mapsto (dq_{\alpha\beta}) \quad{\rm where}\quad 
dq_{\alpha\beta}(x_\beta) = q_\beta(x_\beta) - \sum_{y \in E_{\alpha\setminus \beta}} q_\alpha(x_\beta, y) $$ 

measuring the inconsistency between local beliefs and marginals from their parent faces. _Consistent beliefs_ 
$q \in \Gamma$ are local probability measures $q_\alpha \in {\rm Prob}(E_\alpha)$ indexed by $\alpha \in K$, satisfying the consistency constraint $dq = 0$.

### combinatorial automorphisms

$-$ The fundamental _automorphisms_ $\zeta$ and $\mu$ induced by the order $\supseteq$ on $K$
are returned by `K.zeta(n)` and `K.mu(n)`, where $\zeta^{(n)} : A_n \to A_n$ is inverted by $\mu^{(n)}$.
The zeta transorm, initially introduced in number theory by Dirichlet, and Möbius inversion formulas are related inclusion-exclusion principles in combinatorics. The correspondence $\zeta : A_0 \to A_0$ defined by: 

$$(u_\beta) \mapsto (U_\alpha) \quad{\rm where}\quad U_\alpha(x_\alpha) = \sum_{\beta \subseteq \alpha} u_\beta(x_\beta)$$ 

is therefore invertible by $\mu : A_0 \to A_0$.

## Diffusion

Diffusions are associated to transport equations on potentials $u \in A_0$ for smooth flux functionals 
$\Phi : A_0 \to A_1$ of the form:

$$ \frac {du_\beta} {dt}(x_\beta) = 
\sum_{\alpha \supset \beta} \Phi(u)_{\alpha\beta}(x_\beta) 
- \sum_{\gamma \subset \beta} \Phi(u)_{\beta\gamma}(x_\gamma)$$ 

or written more succintly as $\dot u = \delta \Phi(u)$, 
where the _divergence_ operator $\delta : A_1 \to A_0$ conserves the total energy $\sum_\alpha u_\alpha$. 

Given a time-step $\varepsilon > 0$, diffusion algorithms consist
of iterating $u$ `+=` $\varepsilon \cdot \delta \Phi(u)$ until the local beliefs defined by:
$$ q_\alpha(x_\alpha) = \frac 1 {Z_\alpha} {\rm e}^{-U_\alpha(x_\alpha)}
\quad{\rm where}\quad U_\alpha = \sum_{\beta \subseteq \alpha}u_\beta $$
eventually reach a consistent stationary state $q \in \Gamma$. 

### flux functionals

Different flux functionals $\Phi : A_0 \to A_1$ 
can be used to that end, provided they vanish on the preimage $A_0^\Gamma \subset A_0$ of $\Gamma$, and preferably _only_ on $A_0^\Gamma$. 

Such functionals are generally constructed by compositions of the _effective energy gradient_ 
${\cal D} : A_0 \to A_1$, acting on local hamiltonians $U \in A_0$ by: 

$$ {\cal D} U_{\alpha\beta}(x_\beta) = U_\beta(x_\beta) 
+ \ln \sum_{y \in E_{\alpha \setminus \beta}} {\rm e}^{-U_\alpha(x_\beta, y)} $$

$-$ Bethe $\varepsilon$-diffusion consists of iterating $u$ `+=` $\varepsilon \cdot \delta \phi(u)$ with: 

$$ \phi_{\alpha\beta}(x_\beta) = c_\alpha \Phi_{\alpha\beta}(x_\beta) $$

The Bethe numbers are related to inclusion-exclusion principles on $K$ and eliminate redundant components in
GBP heat flux $\Phi$. Both algorithms are available as `Diffusion` instances from the integrator module. 

The Bethe numbers $c_\beta \in {\mathbb Z}$ are the unique scalar coefficients on $K$ satisfying for all $\beta$: 

$$\sum_{\alpha \supseteq \beta} c_\alpha = 1$$

In [5]:
# K.bethe